In [1]:
import re

file = open('input1.txt', 'r')
regex = '(1|2|3|4|5|6|7|8|9|0|one|two|three|four|five|six|seven|eight|nine)'
translation = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9}

sum_digits = 0
sum_digits_and_text = 0

for line in file.readlines():
    matches = []
    for i in range(len(line)):
        match = re.search(regex, line[i:])
        if match is not None:
            matches.append(match.group(0))

    first = -1
    first_digit = -1

    last = -1
    last_digit = -1

    for match in matches:
        digit = 0
        try:
            digit = int(match)
            if first_digit == -1:
                first_digit = digit
            last_digit = digit
        except ValueError:
            digit = translation[match]

        if first == -1:
            first = digit
        last = digit

    if first == -1 or last == -1:
        print(matches)
        print(first)
        print(last)
        
    sum_digits += 10*first_digit + last_digit
    sum_digits_and_text += 10*first + last

print(f'The sum of the first and last digit on each line is {sum_digits}')
print(f'The sum of the first and last digit on each line (including text) is {sum_digits_and_text}')
        

The sum of the first and last digit on each line is 53386
The sum of the first and last digit on each line (including text) is 53312


In [2]:
import re
file = open('input2.txt')

max_red = 12
max_green = 13
max_blue = 14

impossible = []
possible = []
powers = []

for line in file.readlines():
    game_match = re.match('Game ([0-9]+): (.+)$', line)
    id = int(game_match.group(1))
    draws = game_match.group(2).split(';')

    min_red = 0
    min_green = 0
    min_blue = 0
    
    for draw in draws:
        red_match = re.search('([0-9]+) red', draw)
        green_match = re.search('([0-9]+) green', draw)
        blue_match = re.search('([0-9]+) blue', draw)

        red = 0 if red_match is None else int(red_match.group(1))
        green = 0 if green_match is None else int(green_match.group(1))
        blue = 0 if blue_match is None else int(blue_match.group(1))

        min_red = red if red > min_red else min_red
        min_green = green if green > min_green else min_green
        min_blue = blue if blue > min_blue else min_blue

        if id not in impossible and (red > max_red or green > max_green or blue > max_blue):
            impossible.append(id)

    if id not in impossible:
        possible.append(id)

    powers.append(min_red*min_green*min_blue)

print(f'The sum of the IDs of the games that were possible is {sum(possible)}')
print(f'The sum of the powers of the minimum sets of cubes is {sum(powers)}')
        

The sum of the IDs of the games that were possible is 2406
The sum of the powers of the minimum sets of cubes is 78375


In [3]:
import re
import numpy

file = open('input3.txt')
schematic = []
part_numbers = []

gears = {}

for line in file.readlines():
    schematic.append(line.strip())

running_number = ''
adjacent_gears = set()
adjacent = False

for i in range(len(schematic)):
    for j in range(len(schematic[i])):
        c = schematic[i][j]
        if c.isdigit():
            running_number += c

            # Check nearby cells for symbols
            for k in range(-1, 2):
                for l in range(-1, 2):
                    try:
                        other_c = schematic[i+k][j+l]
                        if re.search('[^\.0-9]', other_c) is not None:
                            adjacent = True
                        if other_c == '*':
                            adjacent_gears.add((i+k, j+l))
                    except IndexError:
                        pass

        else:
            if adjacent:
                part_numbers.append(int(running_number))
                for gear in adjacent_gears:
                    if gear not in gears:
                        gears[gear] = []
                    gears[gear].append(int(running_number))

            running_number = ''
            adjacent_gears = set()
            adjacent = False

print(f'The sum of all the part numbers is {sum(part_numbers)}')
print(f'The sum of all gear ratios is {sum(numpy.prod(gears[gear]) for gear in gears if len(gears[gear]) == 2)}')

The sum of all the part numbers is 539637
The sum of all gear ratios is 82818007


In [4]:
import re

file = open('input4.txt', 'r')
cards = []
for line in file.readlines():
    match = re.search('Card +([0-9]+): +(.*)', line)
    winning_numbers, card_numbers = match.group(2).split('|')

    winning_numbers = winning_numbers.split(' ')
    winning_numbers = [int(number) for number in winning_numbers if number.isdigit()]

    card_numbers = card_numbers.split(' ')
    card_numbers = [int(number) for number in card_numbers if number.isdigit()]

    cards.append({
        'winning_numbers': winning_numbers,
        'card_numbers': card_numbers
    })

points = 0
for card in cards:
    count = 0
    for number in card['card_numbers']:
        if number in card['winning_numbers']:
            count += 1
    if count > 0:
        points += 2**(count-1)

    card['count'] = count

print(f'The sum of scratch card points is {points}')

number_of_cards = [1 for card in cards]
for i in range(len(cards)):
    count = cards[i]['count']
    for j in range(count):
        number_of_cards[i+j+1] += number_of_cards[i]

print(f'The total number of scratch cards at the end is {sum(number_of_cards)}')

The sum of scratch card points is 19135
The total number of scratch cards at the end is 5704953


In [5]:
inputs = open('input5.txt', 'r').read().split('\n\n')

# Start with a map from the seed ranges to itself
seeds = inputs[0].split()[1:]
maps = []
for i in range(0, len(seeds), 2):
    start_range = int(seeds[i])
    length = int(seeds[i+1])
    end_range = start_range + length - 1
    maps.append({'source_range': (start_range, end_range), 'transformation': 0})
    # Add single number ranges to solve part A
    maps.append({'source_range': (length, length), 'transformation': 0})

# Go through the mappings one at a time, and append them to the total map
for mapping in inputs[1:]:
    new_maps = []
    for single_map in mapping.strip().split('\n')[1:]:
        target, source, length = [int(x) for x in single_map.split()]
        source_range = (source, source + length - 1)
        transformation = target - source
        new_maps.append({'source_range': source_range, 'transformation': transformation})

    maps = append_maps(maps, new_maps)

def apply_map(start_value, maps):
    for map in maps:
        if map['source_range'][0] <= start_value <= map['source_range'][1]:
            return start_value + map['transformation']
    return start_value

print(f'The lowest reachable location taking the seeds as discrete locations if {min(apply_map(int(seed), maps) for seed in seeds)}')
print(f'The lowest reachable location is {min(map["source_range"][0] + map["transformation"] for map in maps)}')

# Create a set of maps from seed to location, by appending each intermediate map to the running total of maps
def append_maps(original_maps, appended_maps):
    result_maps = []
    has_intersections = True

    while has_intersections:
        has_intersections = False
        for i in range(len(original_maps)):
            original_map = original_maps[i]
            transformation = original_map['transformation']
            target_range = tuple(x + transformation for x in original_map['source_range'])

            for appended_map in appended_maps:
                intersection, non_intersected_sections = find_intersections(target_range, appended_map['source_range'])
                if intersection is not None:
                    has_intersections = True
                    del original_maps[i]

                    appended_transformation = appended_map['transformation']
                    new_source_range = tuple(x - transformation for x in intersection)
                    result_maps.append({'source_range': new_source_range, 'transformation': transformation + appended_transformation})
                    for section in non_intersected_sections:
                        section_range = tuple(x - transformation for x in section)
                        original_maps.append({'source_range': section_range, 'transformation': transformation})
                    break

            if has_intersections:
                break

    # If anything remains in the original maps list at this point, it had no overlaps with the appended_maps
    for original_map in original_maps:
        result_maps.append(original_map)
    return result_maps

def find_intersections(range1, range2):
    intersection = (max(range1[0], range2[0]), min(range1[1], range2[1]))
    if intersection[1] < intersection[0]:
        intersection = None
    non_intersecting = set()
    if range1[0] < range2[0]:
        non_intersecting.add((range1[0], min(range1[1], range2[0]-1)))
    if range2[1] < range1[1]:
        non_intersecting.add((max(range1[0], range2[1]+1), range1[1]))

    return intersection, non_intersecting

NameError: name 'append_maps' is not defined

In [ ]:
import numpy
import math

def find_wins(time, record):
    return time - 2 * (math.floor((time - math.sqrt(time**2 - 4 * record)) / 2 + 1)) + 1

file = open('input6.txt', 'r')
times_string, distances_string = file.readlines()
times = [int(x) for x in times_string.split() if x.isdigit()]
distances = [int(x) for x in distances_string.split() if x.isdigit()]

wins = [find_wins(time, record) for time, record in zip(times, distances)]
print(f'The product of the numbers of ways to win the short races is {numpy.prod(wins)}')

# One long race
time = int(''.join(x for x in times_string if x.isdigit()))
record = int(''.join(x for x in distances_string if x.isdigit()))
print(f'The number of ways to win the really long race is {find_wins(time, record)}')

In [ ]:
import functools

hands = [line.split(' ') for line in open('input7.txt', 'r').readlines()]
hands = [(x, int(y)) for x,y in hands]

card_values = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
with_joker_card_values = ['A', 'K', 'Q', 'T', '9', '8', '7', '6', '5', '4', '3', '2', 'J']

def get_hand_score(hand):
    cards = {}
    for card in hand[0]:
        if card not in cards:
            cards[card] = 0
        cards[card] += 1

    # Check for five of a kind
    if 5 in cards.values():
        return 19
    if 4 in cards.values():
        return 18
    if 3 in cards.values() and 2 in cards.values():
        return 17
    if 3 in cards.values():
        return 16

    pairs = sum(1 if value == 2 else 0 for value in cards.values())
    if pairs == 2:
        return 15
    if pairs == 1:
        return 14

    return 13

def compare_hands(hand1, hand2):
    comparison = get_hand_score(hand1) - get_hand_score(hand2)

    card_index = 0
    while comparison == 0 and card_index < 5:
        comparison = card_values.index(hand2[0][card_index]) - card_values.index(hand1[0][card_index])
        card_index += 1
    return comparison

def get_joker_hand_score(hand):
    cards = {}
    jokers = 0
    for card in hand[0]:
        if card == 'J':
            jokers += 1
            continue
            
        if card not in cards:
            cards[card] = 0
        cards[card] += 1

    if max(cards.values(), default=0) + jokers == 5:
        return 19
    if max(cards.values(), default=0) + jokers == 4:
        return 18

    pairs = sum(1 if value == 2 else 0 for value in cards.values())
    if (3 in cards.values() and 2 in cards.values()) or (pairs == 2 and jokers == 1):
        return 17

    if max(cards.values(), default=0) + jokers == 3:
        return 16

    if pairs == 2:
        return 15

    if pairs == 1 or jokers == 1:
        return 14

    return 13

def compare_joker_hands(hand1, hand2):
    comparison = get_joker_hand_score(hand1) - get_joker_hand_score(hand2)

    card_index = 0
    while comparison == 0 and card_index < 5:
        comparison = with_joker_card_values.index(hand2[0][card_index]) - with_joker_card_values.index(hand1[0][card_index])
        card_index += 1
    return comparison

hands.sort(key=functools.cmp_to_key(compare_hands))

winnings = 0
for index, hand in enumerate(hands):
    winnings += (index+1) * hand[1]
print(f'The total winnings for the hands is {winnings}')

hands.sort(key=functools.cmp_to_key(compare_joker_hands))

winnings = 0
for index, hand in enumerate(hands):
    winnings += (index+1) * hand[1]
print(f'The total winnings for the hands, including jokers, is {winnings}')

In [ ]:
import re, math
instruction_search = '(\w+) = \((\w+), (\w+)\)'
instructions = open('input8.txt', 'r').read().split('\n')

directions = [0 if c == 'L' else 1 for c in instructions[0]]
nodes = {match.group(1): [match.group(2), match.group(3)] for instruction in instructions if (match := re.search(instruction_search, instruction))}

current_node = 'AAA'
steps = 0

while current_node != 'ZZZ':
    current_node = nodes[current_node][directions[steps % len(directions)]]
    steps += 1
print(f'It took {steps} steps to get to ZZZ')

ghost_nodes = [node for node in nodes if re.match('..A', node)]
steps = 0

# Find cycles
def find_cycle(node):
    current_node = node
    history = []
    current_step = 0

    def find_cycle():
        cycles = 1
        while cycles*len(directions) < len(history):
            if history[-1-cycles*len(directions)] == history[-1]:
                return cycles*len(directions)
            cycles += 1
        return -1

    while (cycle_length := find_cycle()) == -1:
        history.append(current_node)
        current_node = nodes[current_node][directions[current_step % len(directions)]]
        current_step += 1

    # Find the Zs witin the cycle
    targets_in_cycle = []
    for i in range(cycle_length):
        if re.search('..Z', history[-1-cycle_length+i]):
            targets_in_cycle.append(i)

    return cycle_length, current_step - cycle_length - 1, targets_in_cycle

cycles = [find_cycle(ghost_node) for ghost_node in ghost_nodes]

# This task has been made in order to make this easier
lcm = 1
for cycle in cycles:
    lcm = math.lcm(lcm, cycle[0])
print(f'The number of ghost steps to take is {lcm}')

In [ ]:
sequences = [[int(x) for x in line.split()] for line in open('input9.txt', 'r').readlines()]

def reduce_sequence(sequence):
    return [sequence[i] - sequence[i-1] for i in range(1,len(sequence))]

def expand_sequence_backwards(expanding_sequence, lower_sequence):
    if len(lower_sequence) != len(expanding_sequence):
        raise Exception('ME NO WORKY')

    expanded_sequence = [x for x in expanding_sequence]
    expanded_sequence.append(expanding_sequence[-1] + lower_sequence[-1])
    return expanded_sequence

def expand_sequence_forwards(expanding_sequence, lower_sequence):
    if len(lower_sequence) != len(expanding_sequence):
        raise Exception('NO WORKY WORSE')

    expanded_sequence = [x for x in expanding_sequence]
    expanded_sequence.insert(0, expanding_sequence[0] - lower_sequence[0])
    return expanded_sequence

def is_all_zero(sequence):
    for x in sequence:
        if x != 0:
            return False
    return True

expanded_sequences = []
for sequence in sequences:
    altered_sequences = [[x for x in sequence]]
    depth = 0
    while not is_all_zero(altered_sequences[depth]):
        altered_sequences.append(reduce_sequence(altered_sequences[depth]))
        depth += 1

    altered_sequences[depth].append(0)

    for i in range(depth, 0, -1):
        altered_sequences[i - 1] = expand_sequence_backwards(altered_sequences[i - 1], altered_sequences[i])

    altered_sequences[depth].append(0)
    for i in range(depth, 0, -1):
        altered_sequences[i - 1] = expand_sequence_forwards(altered_sequences[i - 1], altered_sequences[i])
    
    expanded_sequences.append(altered_sequences[0])

print(f'The sum of all extrapolated values is {sum(expanded_sequence[-1] for expanded_sequence in expanded_sequences)}')
print(f'The sum of all forward extrapolated values is {sum(expanded_sequence[0] for expanded_sequence in expanded_sequences)}')

In [ ]:
lines = open('input10.txt', 'r').readlines()
connections = {}
start = None

size = (len(lines), len(lines[0]))

for row, line in enumerate(lines):
    for column, character in enumerate(line):
        if character == 'S':
            start = (row, column)
        elif character == '.':
            continue
        elif character == 'F':
            connections[(row, column)] = [(row+1, column), (row, column+1)]
        elif character == '7':
            connections[(row, column)] = [(row+1, column), (row, column-1)]
        elif character == 'J':
            connections[(row, column)] = [(row, column-1), (row-1, column)]
        elif character == 'L':
            connections[(row, column)] = [(row-1, column), (row, column+1)]
        elif character == '-':
            connections[(row, column)] = [(row, column-1), (row, column+1)]
        elif character == '|':
            connections[(row, column)] = [(row-1, column), (row+1, column)]

start_connections = []
for i in range(-1, 2, 1):
    for j in range(-1, 2, 1):
        test_coordinate = (start[0] + i, start[1] + j)
        if test_coordinate in connections and start in connections[test_coordinate]:
            start_connections.append(test_coordinate)

# Replace the start icon
above = (start[0]-1, start[1])
right = (start[0], start[1]+1)
below = (start[0]+1, start[1])
left = (start[0], start[1]-1)

if above in start_connections and right in start_connections:
    start_sign = 'L'
elif above in start_connections and below in start_connections:
    start_sign = '|'
elif above in start_connections and left in start_connections:
    start_sign = 'J'
elif right in start_connections and below in start_connections:
    start_sign = 'F'
elif right in start_connections and left in start_connections:
    start_sign = '-'
elif below in start_connections and left in start_connections:
    start_sign = '7'

loop = set()
current = start_connections[0]
last = start
steps = 1

loop.add(start)
loop.add(current)

while current != start:
    for connection in connections[current]:
        if connection != last:
            last = current
            current = connection
            loop.add(current)
            steps += 1
            break
print(f'The farthest point from the starting position is {steps//2} steps away')

enclosed = 0
last_hit = None
for row in range(size[0]):
    inside = False
    for column in range(size[1]):
        if (row, column) in loop:
            c = lines[row][column]
            if c == 'S':
                c = start_sign
            
            if c == '|':
                inside = not inside
            if c == 'F' or c == 'L':
                last_hit = c
            if c == '7' and last_hit == 'L':
                inside = not inside
            if c == 'J' and last_hit == 'F':
                inside = not inside
        elif inside:
            enclosed += 1

print(f'There are {enclosed} tiles enclosed within the loop')

In [ ]:
import itertools

input = open('input11.txt', 'r').readlines()
map = []
galaxy_counter = 1
for line in input:
    map_line = []
    for c in line:
        if c == '.':
            map_line.append(0)
        elif c == '#':
            map_line.append(galaxy_counter)
            galaxy_counter += 1
    map.append(map_line)

locations = {}
for row_index, row in enumerate(map):
    for column_index, column in enumerate(row):
        if column != 0:
            locations[int(column)] = (row_index, column_index)

empty_rows = [index for index, row in enumerate(map) if sum(row) == 0]
no_of_columns = len(map[0])
empty_columns = [i for i in range(no_of_columns) if sum(row[i] for row in map) == 0]
no_of_columns += len(empty_columns)

# Add empty columns
def expand(locations, expanded_rows, expanded_columns, expansion_size):
    expanded_locations = {}
    for location in locations:
        coordinate = locations[location]
        row_expansions = sum(1 if row < coordinate[0] else 0 for row in expanded_rows)
        column_expansions = sum(1 if column < coordinate[1] else 0 for column in expanded_columns)

        expanded_locations[location] = (coordinate[0] + expansion_size * row_expansions, coordinate[1] + expansion_size * column_expansions)
    return expanded_locations

all_pairs = [(i,j) for i in range(1,len(locations)+1) for j in range(i+1, len(locations)+1)]
small_expansion = expand(locations, empty_rows, empty_columns, 1)
large_expansion = expand(locations, empty_rows, empty_columns, 999999)

def find_distance_between_pair(pair, locations):
    location_one = locations[pair[0]]
    location_two = locations[pair[1]]

    return abs(location_one[0] - location_two[0]) + abs(location_one[1] - location_two[1])

print(f'The sum of distances between all galaxy pairs is {sum(find_distance_between_pair(pair, small_expansion) for pair in all_pairs)}')
print(f'The sum of distances between all galaxy pairs in the greatly expanded universe is {sum(find_distance_between_pair(pair, large_expansion) for pair in all_pairs)}')

In [ ]:
input = open('input12.txt', 'r').readlines()

def find_combinations(string, runs, seen):
    if len(runs) == 0:
        if '#' in string:
            return 0
        else:
            return 1

    combinations = 0
    run_length = runs[0]
    for i in range(len(string) - run_length+1):
        if i > 0 and string[i-1] == '#':
            break
        test_string = string[i:i+run_length+1]
        if '.' not in test_string[:run_length] and (len(test_string) == run_length or test_string[-1] != '#'):
            next_string = string[i+run_length+1:]
            next_runs = runs[1:]
            if (next_string,) + tuple(next_runs) not in seen:
                this_combinations = find_combinations(next_string, next_runs, seen)
                seen[(next_string,) + tuple(next_runs)] = this_combinations
            else:
                this_combinations = seen[(next_string,) + tuple(next_runs)]
            combinations += this_combinations

    return combinations

number_of_arrangements = []
number_of_unfolded_arrangements = []
for line_index, line in enumerate(input):
    string, numbers = line.split()
    runs = [int(x) for x in numbers.split(',')]
    seen = {}
    number_of_combinations = find_combinations(string, runs, seen)
    number_of_arrangements.append(number_of_combinations)

    long_string = ((string + '?')*5)[:-1]
    long_runs = runs*5

    number_of_combinations = find_combinations(long_string, long_runs, seen)
    number_of_unfolded_arrangements.append(number_of_combinations)
    print(f'\rOn line {line_index+1} of {len(input)}', end='')

print(f'\nThe total number of arrangements over all strings is {sum(number_of_arrangements)}')
print(f'The total number of unfolded arrangements is {sum(number_of_unfolded_arrangements)}')

In [ ]:
areas = open('input13.txt', 'r').read().split('\n\n')
vertical_lines = 0
horizontal_lines = 0

smudge_vertical_lines = 0
smudge_horizontal_lines = 0

def find_horizontal_reflection(lines, old_line=None):
    for i in range(1, len(lines)):
        reflected = True
        for j in range(i):
            if i+j >= len(lines):
                break
            if lines[i-j-1] != lines[i+j]:
                reflected = False
                break

        if reflected and (old_line is None or old_line != i):
            return i

    return None

def find_area_reflections(area, old_horizontal=None, old_vertical = None):
    area_lines = area.strip().split('\n')
    horizontal = find_horizontal_reflection(area_lines, old_horizontal)
    flipped = [''.join(line[i] for line in area_lines) for i in range(len(area_lines[0]))]
    vertical = find_horizontal_reflection(flipped, old_vertical)
    return horizontal, vertical

for area in areas:
    horizontal, vertical = find_area_reflections(area)
    if horizontal is not None:
        horizontal_lines += horizontal
    if vertical is not None:
        vertical_lines += vertical

    # Smudge correction
    found = False
    for i in range(len(area)):
        new_area = None
        if area[i] == '.':
            new_area = area[:i] + '#' + area[i+1:]
        elif area[i] == '#':
            new_area = area[:i] + '.' + area[i+1:]
        else:
            continue

        new_horizontal, new_vertical = find_area_reflections(new_area, horizontal, vertical)
        if new_horizontal is not None:
            smudge_horizontal_lines += new_horizontal
            found = True
            break
        if new_vertical is not None:
            smudge_vertical_lines += new_vertical
            found = True
            break
    if not found:
        print(area)
        print()

print(f'The solution number is {100*horizontal_lines + vertical_lines}')
print(f'The solution number when correcting for smudges is {100*smudge_horizontal_lines + smudge_vertical_lines}')

In [ ]:
input = '''O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....'''.split('\n')
input = open('input14.txt', 'r').readlines()
platform_height = len(input)
platform_width = len(input[0].strip())
solid = set()
round = set()
for row, line in enumerate(input):
    for column, character in enumerate(line):
        if character == '#':
            solid.add((row, column))
        elif character == 'O':
            round.add((row, column))

def roll_north(round):
    rolled = set()
    for j in range(platform_width):
        last_solid = -1
        no_rocks = 0
        for i in range(platform_height):
            if (i, j) in round:
                no_rocks += 1
            if (i,j) in solid:
                for k in range(no_rocks):
                    rolled.add((last_solid+k+1, j))
                last_solid = i
                no_rocks = 0
        for k in range(no_rocks):
            rolled.add((last_solid+k+1, j))
    return rolled

def roll_west(round):
    rolled = set()
    for i in range(platform_height):
        last_solid = -1
        no_rocks = 0
        for j in range(platform_width):
            if (i, j) in round:
                no_rocks += 1
            if (i, j) in solid:
                for k in range(no_rocks):
                    rolled.add((i, last_solid + k + 1))
                last_solid = j
                no_rocks = 0
        for k in range(no_rocks):
            rolled.add((i, last_solid+k+1))
    return rolled

def roll_south(round):
    rolled = set()
    for j in range(platform_width):
        last_solid = platform_height
        no_rocks = 0
        for i in range(platform_height-1, -1, -1):
            if (i, j) in round:
                no_rocks += 1
            if (i, j) in solid:
                for k in range(no_rocks):
                    rolled.add((last_solid-k-1, j))
                no_rocks = 0
                last_solid = i
        for k in range(no_rocks):
            rolled.add((last_solid-k-1, j))
    return rolled

def roll_east(round):
    rolled = set()
    for i in range(platform_height):
        no_rocks = 0
        last_solid = platform_width
        for j in range(platform_width-1, -1, -1):
            if (i, j) in round:
                no_rocks += 1
            if (i, j) in solid:
                for k in range(no_rocks):
                    rolled.add((i, last_solid-k-1))
                no_rocks = 0
                last_solid = j
        for k in range(no_rocks):
            rolled.add((i, last_solid-k-1))
    return rolled

def calculate_load(round):
    total_load = 0
    for rock in round:
        total_load += platform_height - rock[0]
    return total_load

new_round = roll_north(roll_south(round))
total_load = calculate_load(new_round)
print(f'The total load on the platform is {total_load}')

cycle_start = None
cycle_length = None
seen = [round]

for i in range(1000000000):
    round = roll_east(roll_south(roll_west(roll_north(round))))
    if round in seen:
        cycle_start = seen.index(round)
        cycle_length = i + 1 - cycle_start
        break
    seen.append(round)

print(f'After 1000000000 cycles, the load is {calculate_load(seen[(1000000000 - cycle_start) % cycle_length + cycle_start])}')


In [ ]:
import re
input = open('input15.txt', 'r').read().replace('\n','').split(',')

def hash_string(string):
    return sum(ord(c)*17**(i+1) for i, c in enumerate(string[::-1])) % 256

boxes = {i: {} for i in range(256)}
for label, focal_length in [re.split('[-=]', line) for line in input]:
    box_no = hash_string(label)
    if focal_length != '':
        boxes[box_no][label] = int(focal_length)
    elif label in boxes[box_no]:
        del boxes[box_no][label]
        
focusing_power = sum((box_no + 1) * (label_no + 1) * boxes[box_no][label] for box_no in boxes for label_no, label in enumerate(boxes[box_no]))
        
print(f'The sum of the hashes of each step is {sum(hash_string(step) for step in input)}')
print(f'The total focusing power is {focusing_power}')

In [ ]:
input_lines = open('input16.txt', 'r').readlines()
height = len(input_lines)
width = len(input_lines[0].strip())

beam_transformers = {}

def move_beam(beam, direction=None):
    beam_direction = beam['direction']
    if direction is not None:
        beam_direction = direction

    beam_position = beam['position']
    new_position = (beam_position[0] + beam_direction[0], beam_position[1] + beam_direction[1])
    beam['position'] = new_position
    beam['direction'] = beam_direction

def create_splitter(character):
    if character == '-':
        def splitter(beam):
            if beam['direction'] == (0, 1) or beam['direction'] == (0, -1):
                new_beams = [{'position': beam['position'], 'direction': (1, 0)}, {'position': beam['position'], 'direction': (-1, 0)}]
                for new_beam in new_beams:
                    move_beam(new_beam)
                return new_beams
            else:
                move_beam(beam)
                return [beam]
        return splitter
    elif character == '|':
        def splitter(beam):
            if beam['direction'] == (1, 0) or beam['direction'] == (-1, 0):
                new_beams = [{'position': beam['position'], 'direction': (0, 1)}, {'position': beam['position'], 'direction': (0, -1)}]
                for new_beam in new_beams:
                    move_beam(new_beam)
                return new_beams
            else:
                move_beam(beam)
                return [beam]
        return splitter

def create_mirror(character):
    if character == '\\':
        def mirror(beam):
            if beam['direction'] == (1, 0):
                new_beam = {'position': beam['position'], 'direction': (0, 1)}
            elif beam['direction'] == (-1, 0):
                new_beam = {'position': beam['position'], 'direction': (0, -1)}
            elif beam['direction'] == (0, 1):
                new_beam = {'position': beam['position'], 'direction': (1, 0)}
            elif beam['direction'] == (0, -1):
                new_beam = {'position': beam['position'], 'direction': (-1, 0)}
            else:
                new_beam = beam
            move_beam(new_beam)
            return [new_beam]
        return mirror

    elif character == '/':
        def mirror(beam):
            if beam['direction'] == (1, 0):
                new_beam = {'position': beam['position'], 'direction': (0, -1)}
            elif beam['direction'] == (-1, 0):
                new_beam = {'position': beam['position'], 'direction': (0, 1)}
            elif beam['direction'] == (0, 1):
                new_beam = {'position': beam['position'], 'direction': (-1, 0)}
            elif beam['direction'] == (0, -1):
                new_beam = {'position': beam['position'], 'direction': (1, 0)}
            else:
                new_beam = beam
            move_beam(new_beam)
            return [new_beam]
        return mirror

for x in range(width):
    for y in range(height):
        if input_lines[y][x] == '-' or input_lines[y][x] == '|':
            beam_transformers[(x, y)] = create_splitter(input_lines[y][x])
        elif input_lines[y][x] == '\\' or input_lines[y][x] == '/':
            beam_transformers[(x,y)] = create_mirror(input_lines[y][x])

def run_beam(beam):
    beams = [beam]
    seen_before = set()
    energized = set()
    
    while len(beams) > 0:
        # Remove beams outside the board
        for i in range(len(beams)-1, -1, -1):
            beam = beams[i]
            position = beam['position']
            if position[0] < 0 or position[0] >= width or position[1] < 0 or position[1] >= height:
                del beams[i]
        # Remove beams with positions and directions we have seen before
        # This will also remove duplicates
        for i in range(len(beams)-1, -1, -1):
            beam = beams[i]
            position = beam['position']
            direction = beam['direction']
    
            if (position, direction) in seen_before:
                del beams[i]
            else:
                seen_before.add((position, direction))
    
        # Energize tiles
        for beam in beams:
            energized.add(beam['position'])
    
        # Move beams
        for i in range(len(beams)-1, -1, -1):
            beam = beams[i]
            position = beam['position']
            if position in beam_transformers:
                del beams[i]
                for new_beam in beam_transformers[position](beam):
                    beams.append(new_beam)
            else:
                move_beam(beam)
    return len(energized)

activations = []
for i in range(2*height + 2*width):
    if i < height:
        beam = {'position': (0, i), 'direction': (1, 0)}
    elif height <= i < height + width:
        beam = {'position': (i - height, height - 1), 'direction': (0, -1)}
    elif height + width <= i < 2*height + width:
        beam = {'position': (width-1, i - width - height), 'direction': (-1, 0)}
    elif 2*height + width <= i:
        beam = {'position': (i - 2*height - width, 0), 'direction': (0, 1)}

    energization = run_beam(beam)
    if i == 0:
        print(f'The initial beam energized {energization} tiles')
    activations.append(energization)
print(f'The best beam activizes {max(activations)} tiles')        

In [1]:
test_input = '''2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533'''

maze = [[int(x) for x in line.strip()] for line in test_input.split('\n')]
maze = [[int(x) for x in line.strip()] for line in open('input17.txt', 'r').readlines()]

width = len(maze[0])
height = len(maze)

directions = ['HORIZONTAL', 'VERTICAL']

def find_neighbours(x, y, allowed_directions=None, min_range=1, max_range=1):
    neighbours = []
    if allowed_directions is None:
        allowed_directions = directions
    for direction in allowed_directions:
        plus_cost = 0
        minus_cost = 0
        for i in range(1, max_range+1):
            if direction == 'HORIZONTAL':
                plus_x = x+i
                plus_y = y
                minus_x = x-i
                minus_y = y
            if direction == 'VERTICAL':
                plus_x = x
                plus_y = y+i
                minus_x = x
                minus_y = y-i

            if plus_x < width and plus_y < height:
                plus_cost += maze[plus_y][plus_x]
                if i >= min_range:
                    neighbours.append((plus_x, plus_y, plus_cost, direction))
            if minus_x >= 0 and minus_y >= 0:
                minus_cost += maze[minus_y][minus_x]
                if i >= min_range:
                    neighbours.append((minus_x, minus_y, minus_cost, direction))
    return neighbours

def find_path(start_x, start_y, use_directions=False, goal_x=None, goal_y=None, min_range=1, max_range=1, heuristic=None):
    start = None

    if use_directions:
        start = (start_x, start_y, None)
    else:
        start = (start_x, start_y)

    candidates = {start: 0}
    visited = {}

    while len(candidates) > 0:
        minimum = 1e100
        best_candidate = None
        for candidate in candidates:
            cost = candidates[candidate]
            if heuristic is not None:
                cand_x = candidate[0]
                cand_y = candidate[1]
                cost += heuristic(cand_x, cand_y)

            if cost < minimum:
                minimum = cost
                best_candidate = candidate

        cost = candidates[best_candidate]
        x = best_candidate[0]
        y = best_candidate[1]
    
        del candidates[best_candidate]
        visited[best_candidate] = cost

        if x == goal_x and y == goal_y:
            break
    
        neighbours = []
        if use_directions:
            previous_direction = best_candidate[2]
            allowed_directions = [direction for direction in directions if direction != previous_direction]
            neighbours = find_neighbours(x, y, allowed_directions, min_range, max_range)
        else:
            neighbours = find_neighbours(x, y, directions, min_range, max_range)
    
        for nx, ny, ncost, ndir in neighbours:
            full_cost = cost + ncost
            neighbour = None
            if use_directions:
                neighbour = (nx, ny, ndir)
            else:
                neighbour = (nx, ny)
                
            if neighbour in visited and visited[neighbour] <= full_cost:
                continue
            if neighbour in candidates and candidates[neighbour] <= full_cost:
                continue
    
            candidates[neighbour] = full_cost

    return visited

goal_point = (width-1, height-1)

# Create a heuristic for A* algorithm
heuristic_array = find_path(*goal_point)
def heuristic(x, y):
    return heuristic_array[(x,y)] - maze[y][x] + maze[goal_point[1]][goal_point[0]]

best_paths = find_path(0, 0, True, *goal_point, 1, 3)
for best_path in best_paths:
    point = (best_path[0], best_path[1])
    if point == goal_point:
        print(f'The fastest way to the goal is {best_paths[best_path]}')

best_paths = find_path(0, 0, True, *goal_point, 4, 10)
for best_path in best_paths:
    point = (best_path[0], best_path[1])
    if point == goal_point:
        print(f'The fastest way to the goal is {best_paths[best_path]}')

The fastest way to the goal is 665
The fastest way to the goal is 809


In [114]:
import re

instructions = '''R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)'''.split('\n')
instructions = open('input18.txt', 'r').readlines()

def move_coordinates(x, y, prev, cur):
    if prev == 'U' and cur == 'R':
        return x-0.5, y+0.5
    if prev == 'U' and cur == 'U':
        return x-0.5, y
    if prev == 'U' and cur == 'L':
        return x-0.5, y-0.5
    if prev == 'L' and cur == 'U':
        return x-0.5, y-0.5
    if prev == 'L' and cur == 'L':
        return x, y-0.5
    if prev == 'L' and cur == 'D':
        return x+0.5, y-0.5
    if prev == 'D' and cur == 'L':
        return x+0.5, y-0.5
    if prev == 'D' and cur == 'D':
        return x+0.5, y
    if prev == 'D' and cur == 'R':
        return x+0.5, y+0.5
    if prev == 'R' and cur == 'D':
        return x+0.5, y+0.5
    if prev == 'R' and cur == 'R':
        return x, y+0.5
    if prev == 'R' and cur == 'U':
        return x-0.5, y+0.5
    else:
        print(prev, cur)

def calculate_volume(instructions):
    volume = 0
    position = (0, 0)
    previous_direction = None
    for i in range(len(instructions)):
        direction, length = instructions[i]
        prev_direction, _ = instructions[(i-1) % len(instructions)]
        next_direction, _ = instructions[(i+1) % len(instructions)]
        x, y = position
        if direction == 'R':
            new_x = x + length
            new_y = y
        elif direction == 'D':
            new_x = x
            new_y = y - length
        elif direction == 'L':
            new_x = x - length
            new_y = y
        elif direction == 'U':
            new_x = x
            new_y = y + length

        vol_x_1, vol_y_1 = move_coordinates(x, y, prev_direction, direction)
        vol_x_2, vol_y_2 = move_coordinates(new_x, new_y, direction, next_direction)

        volume += vol_x_1 * vol_y_2 - vol_y_1 * vol_x_2
        position = (new_x, new_y)
    return volume / 2

easy_instructions = []
for instruction in instructions:
    regmatch = re.search('(.) ([0-9]+)', instruction)
    easy_instructions.append((regmatch.group(1), int(regmatch.group(2))))

hard_instructions = []
direction_map = {'0': 'R', '1': 'D', '2': 'L', '3': 'U'}
for instruction in instructions:
    regmatch = re.search('#([0-9a-f]{5})([0-3])', instruction)
    hard_instructions.append((direction_map[regmatch.group(2)], int(regmatch.group(1), 16)))

print(f'The volume of the lava lagoon is {abs(calculate_volume(easy_instructions))}')
print(f'The volume of the much larger lava lagoon is {abs(calculate_volume(hard_instructions))}')

The volume of the lava lagoon is 72821.0
The volume of the much larger lava lagoon is 127844509405501.0


In [12]:
import re, itertools

input = open('input19.txt', 'r').read()
workflows_input, parts_input = input.split('\n\n')

parts = []
workflows = {}

accepted = []
rejected = []

for line in parts_input.strip().split('\n'):
    part = {}
    part['x'] = int(re.search('x=([0-9]+),', line).group(1))
    part['m'] = int(re.search('m=([0-9]+),', line).group(1))
    part['a'] = int(re.search('a=([0-9]+),', line).group(1))
    part['s'] = int(re.search('s=([0-9]+)}', line).group(1))
    parts.append(part)

for line in workflows_input.strip().split('\n'):
    workflow_match = re.search('(.+){(.+)}', line)
    workflow_name = workflow_match.group(1)
    workflow_instructions = workflow_match.group(2)
    instructions = [x.strip() for x in workflow_instructions.split(',')]
    workflows[workflow_name] = instructions

while len(parts) > 0:
    part = parts[0]
    del parts[0]

    workflow = 'in'
    instruction_number = 0

    while workflow != '':
        instruction = workflows[workflow][instruction_number]
        goal = instruction
        conditional = re.search('(.)(>|<)([0-9]+):(.+)', instruction)
        passed = True
        if conditional is not None:
            variable = conditional.group(1)
            number = int(conditional.group(3))
            goal = conditional.group(4)
            passed = (part[variable] > number) if conditional.group(2) == '>' else (part[variable] < number)
        if passed:
            if goal == 'A':
                accepted.append(part)
                workflow = ''
            elif goal == 'R':
                rejected.append(part)
                workflow = ''
            else:
                workflow = goal
                instruction_number = 0
        else:
            instruction_number += 1

def sum_part(part):
    return part['x'] + part['m'] + part['a'] + part['s']

print(f'The sum of all ratings of accepted parts is {sum(sum_part(part) for part in accepted)}')

accepted_ranges = []
start_range = {'x': (1, 4000), 'm': (1, 4000), 'a': (1, 4000), 's': (1, 4000)}

def find_range_overlap(range_one, range_two):
    overlap = (max(range_one[0], range_two[0]), min(range_one[1], range_two[1]))
    if overlap[1] - overlap[0] < 0:
        return None
    return overlap

def split_range(number, operator, range):
    passing = None
    failing = None
    if operator == '>':
        passing = (number+1, 4000)
        failing = (1, number)
    elif operator == '<':
        passing = (0, number-1)
        failing = (number, 4000)

    return find_range_overlap(passing, range), find_range_overlap(failing, range)

def find_accepted_range(range, workflow):
    ranges = [range]
    for instruction in workflows[workflow]:
        goal = instruction
        conditional = re.search('(.)(>|<)([0-9]+):(.+)', instruction)
        
        if conditional is not None:
            variable = conditional.group(1)
            operator = conditional.group(2)
            number = int(conditional.group(3))
            goal = conditional.group(4)
        else:
            variable = 'x'
            operator = '>'
            number = -1
        
        new_ranges = []
        for old_range in ranges:
            passing, failing = split_range(number, operator, old_range[variable])
            if passing is not None:
                child_range = {x: old_range[x] for x in old_range}
                child_range[variable] = passing

                if goal == 'A':
                    accepted_ranges.append(child_range)
                elif goal != 'R':
                    find_accepted_range(child_range, goal)

            if failing is not None:
                child_range = {x: old_range[x] for x in old_range}
                child_range[variable] = failing
                new_ranges.append(child_range)

        ranges = new_ranges

find_accepted_range(start_range, 'in')
acceptable_combinations = 0
for i in range(len(accepted_ranges)):
    current_range = accepted_ranges[i]
    current_combinations = 1
    for variable in current_range:
        current_combinations *= (current_range[variable][1] + 1 - current_range[variable][0])
    acceptable_combinations += current_combinations

duplicates = 0
for i in range(len(accepted_ranges)):
    for j in range(i+1, len(accepted_ranges)):
        range_one = accepted_ranges[i]
        range_two = accepted_ranges[j]

        overlaps = 1
        for variable in range_one:
            overlapping = find_range_overlap(range_one[variable], range_two[variable])
            if overlapping is None:
                overlaps = 0
                break
            overlaps *= (overlapping[1] - overlapping[0] + 1)
        duplicates += overlaps

print(f'The number of acceptable combinations is {acceptable_combinations - duplicates}')

The sum of all ratings of accepted parts is 575412
The number of acceptable combinations is 126107942006821


In [21]:
import re
import math

input = '''broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output'''.split('\n')
input = open('input20.txt', 'r').readlines()

modules = {}

module_matcher = '(&|%){0,1}(.+) -> (.+)'

# Make a pass to create all the modules
for line in input:
    matcher = re.search(module_matcher, line)
    if matcher is not None:
        type = matcher.group(1)
        name = matcher.group(2)
        connections = matcher.group(3).split(',')
        module = {'type': type}
        modules[name] = module
        for connection in connections:
            if connection.strip() not in modules:
                modules[connection.strip()] = {}

# Create connections
for line in input:
    matcher = re.search(module_matcher, line)
    if matcher is not None:
        name = matcher.group(2)
        connecting = modules[name]
        connecting['connections'] = []
        connections = matcher.group(3).split(',')
        for connection in connections:
            connection = connection.strip()
            connected = modules[connection]
            connecting['connections'].append(connection)
            if 'type' in connected and connected['type'] == '&':
                if 'incoming' not in connected:
                    connected['incoming'] = []
                connected['incoming'].append(name)

# Set up modules properly
for module_name in modules:
    module = modules[module_name]

    if 'type' in module and module['type'] == '%':
        module['state'] = False
        def receive_signal(transmitter, signal, module=module, module_name=module_name):
            transmitted_signals = []
            if signal:
                return transmitted_signals
            for connection in module['connections']:
                transmitted_signals.append((module_name, not module['state'], connection))
            module['state'] = not module['state']
            return transmitted_signals
        module['function'] = receive_signal

    elif 'type' in module and module['type'] == '&':
        module['memory'] = {}
        for incoming in module['incoming']:
            module['memory'][incoming] = False
        def receive_signal(transmitter, signal, module=module, module_name=module_name):
            transmitted_signals = []
            module['memory'][transmitter] = signal
            memory_high = all(module['memory'][incoming] for incoming in module['incoming'])
            for connection in module['connections']:
                transmitted_signals.append((module_name, not memory_high, connection))
            return transmitted_signals
        module['function'] = receive_signal

    elif 'connections' in module:
        def receive_signal(transmitter, signal, module=module, module_name=module_name):
            transmitted_signals = []
            for connection in module['connections']:
                transmitted_signals.append((module_name, signal, connection))
            return transmitted_signals
        module['function'] = receive_signal
    else:
        def receive_signal(transmitter, signal, module=module, module_name=module_name):
            return []
        module['function'] = receive_signal

def press_button():
    signals = [(None, False, 'broadcaster')]
    counter = 0
    while counter < len(signals):
        transmitter, signal, receiver = signals[counter]
        counter += 1

        forwarded_signals = modules[receiver]['function'](transmitter, signal)
        for forwarded_signal in forwarded_signals:
            signals.append(forwarded_signal)
    return signals

def reset():
    for module in modules:
        if 'state' in modules[module]:
            modules[module]['state'] = False
        if 'memory' in modules[module]:
            for remembered in modules[module]['memory']:
                modules[module]['memory'][remembered] = False

for i in range(1000):
    press_button()
reset()

high = 0
low = 0
for i in range(1000):
    signals = press_button()
    high += sum(1 for signal in signals if signal[1])
    low += sum(1 for signal in signals if not signal[1])

print(f'After 1000 button presses, the product of high and low signals is {high*low}')

def count_rx_signals(signals):
    return sum(1 for signal in signals if signal[2] == 'rx' and not signal[1]), sum(1 for signal in signals if signal[2] == 'rx' and signal[1])

def find_cycles(receiving):
    reset()
    on = {}
    for i in range(10000):
        signals = press_button()
        investigated = [signal for signal in signals if signal[2] == receiving]
        for index, (transmitter, signal, receiver) in enumerate(investigated):
            if receiver != receiving:
                continue
            if transmitter not in on:
                on[transmitter] = []
            if signal:
                on[transmitter].append(i+1)
    return on
    

cycles = find_cycles('vr')
for transmitter in cycles:
    start = cycles[transmitter][0]
    period = -1
    for i in range(1,len(cycles[transmitter])):
        length = cycles[transmitter][i] - cycles[transmitter][i-1]
        if period == -1:
            period = length
        elif period != length:
            print('NOT CYCLICAL')
    cycles[transmitter] = {'start': start, 'period': period}

def find_pairwise_cycle(cycle_one, cycle_two):
    x0 = cycle_one['start']
    x1 = cycle_one['period']
    y0 = cycle_two['start']
    y1 = cycle_two['period']
    a = 0
    while (x0 - y0 + a*x1) % y1 != 0:
        a += 1
    b = (x0 - y0 + a*x1) / y1

    return {'start': x0 + a*x1, 'period': math.lcm(x1, y1)}

cycle = {'start': 0, 'period': 1}
for transmitter in cycles:
    cycle = find_pairwise_cycle(cycle, cycles[transmitter])
print(cycles)
print(cycle)

After 1000 button presses, the product of high and low signals is 747304011
{'dr': {'start': 3821, 'period': 3821}, 'tn': {'start': 3761, 'period': 3761}, 'bm': {'start': 3889, 'period': 3889}, 'cl': {'start': 3943, 'period': 3943}}
{'start': 0, 'period': 220366255099387}


In [3]:
import numpy, itertools, math
map = open('input21.txt', 'r').readlines()

steps = 64
long_steps = 26501365

height = len(map)
width = len(map[0].strip())

sx, sy = 0, 0

rocks = set()
for row, line in enumerate(map):
    for column, c in enumerate(line):
        if c == '#':
            rocks.add((column, row))
        if c == 'S':
            sx, sy = column, row

# Good ole' Dijkstra
def perform_dijkstra(sx, sy):
    directions = [(1,0), (0,1), (-1,0), (0, -1)]
    candidates = {(sx, sy): 0}
    visited = {}
    while len(candidates) > 0:
        best_candidate = min(candidates, key=lambda c: candidates[c])
        distance = candidates[best_candidate]
        del candidates[best_candidate]
        visited[best_candidate] = distance
    
        x, y = best_candidate
        
        for direction in directions:
            dx, dy = direction
            nx, ny = x+dx, y+dy
            if (nx, ny) in rocks:
                continue
            if nx < 0 or nx >= width or ny < 0 or ny >= height:
                continue
    
            if (nx, ny) in visited:
                continue
    
            if (nx, ny) in candidates and candidates[(nx, ny)] <= distance+1:
                continue
                
            candidates[(nx, ny)] = distance + 1
    return visited

visited = perform_dijkstra(sx, sy)
parity = steps % 2
print(f'In {steps} steps, the elf can reach {sum(1 for plot in visited if visited[plot] % 2 == parity and visited[plot] <= steps)} garden plots')

corners = {
    'top_left': (0,0),
    'bottom_left': (0, height-1),
    'bottom_right': (width-1, height-1),
    'top_right': (width-1, 0)
}

straights = {
    'left': (0, sy),
    'right': (width-1, sy),
    'top': (sx, 0),
    'bottom': (sx, height-1)
}

opposites = {
    'top_left': 'bottom_right',
    'left': 'right',
    'bottom_left': 'top_right',
    'bottom': 'top',
    'bottom_right': 'top_left',
    'right': 'left',
    'top_right': 'bottom_left',
    'top': 'bottom'
}

distances = {corner: perform_dijkstra(*corners[corner]) for corner in corners}
for straight in straights:
    distances[straight] = perform_dijkstra(*straights[straight])
reachable_in_square = [sum(1 for plot in visited if visited[plot] % 2 == i) for i in range(2)]

parity = long_steps % 2
reachable = reachable_in_square[parity]

for direction in straights:
    remaining = long_steps - visited[straights[opposites[direction]]] - 1
    farthest = max(distances[direction].values())
    fully_covered_squares = (remaining - farthest) // width + 1

    opposite_parity = math.ceil(fully_covered_squares / 2)
    same_parity = fully_covered_squares // 2

    reachable += opposite_parity * reachable_in_square[(parity + 1) % 2] + same_parity * reachable_in_square[parity]
    remaining -= fully_covered_squares*width

    while remaining >= 0:
        remaining_parity = remaining % 2
        cur_distances = distances[direction]
        reachable += sum(1 for plot in cur_distances if cur_distances[plot] <= remaining and cur_distances[plot] % 2 == remaining_parity)
        remaining -= width

for corner in corners:
    remaining = long_steps - visited[corners[opposites[corner]]] - 2
    farthest = max(distances[corner].values())
    farthest_square_distance = (remaining - farthest) // width + 1

    same_parity = math.ceil(farthest_square_distance / 2) ** 2
    opposite_parity = (farthest_square_distance // 2) ** 2 + (farthest_square_distance // 2)
    reachable += same_parity * reachable_in_square[parity] + opposite_parity * reachable_in_square[(parity + 1) % 2]

    remaining -= farthest_square_distance * width
    extra_squares = 0
    while remaining >= 0:
        extra_squares += 1
        remaining_parity = remaining % 2
        reachable_in_current_square = sum(1 for plot in distances[corner] if distances[corner][plot] <= remaining and distances[corner][plot] % 2 == remaining_parity)
        reachable += (farthest_square_distance + extra_squares) * reachable_in_current_square
        remaining -= width

print(f'In {long_steps} steps, the elf can reach {reachable} plots')
    

In 64 steps, the elf can reach 3830 garden plots
In 26501365 steps, the elf can reach 637087163925555 plots


In [5]:
import re
brick_lines = open('input22.txt', 'r').readlines()

bricks = {}
brick_names = []

for index, brick_line in enumerate(brick_lines):
    brick_match = re.search('([0-9]+),([0-9]+),([0-9]+)~([0-9]+),([0-9]+),([0-9]+)', brick_line)
    start_x, end_x = int(brick_match.group(1)), int(brick_match.group(4))
    start_y, end_y = int(brick_match.group(2)), int(brick_match.group(5))
    start_z, end_z = int(brick_match.group(3)), int(brick_match.group(6))

    blocks = [(x, y, z) for x in range(start_x, end_x+1) for y in range(start_y, end_y+1) for z in range(start_z, end_z+1)]
    bricks[index] = {'blocks': blocks, 'supporting': [], 'supported_by': []}
    brick_names.append(index)

brick_names.sort(key=lambda bn: min(z for x, y, z in bricks[bn]['blocks']))

for index, brick_name in enumerate(brick_names):
    horizontal_coverage = set((x, y) for x, y, z in bricks[brick_name]['blocks'])
    # Let bricks fall to either the ground, or another brick
    fall = None
    for other_index, other_brick in enumerate(brick_names):
        # All bricks that we can land on must start under the current brick
        if other_index >= index:
            break
        lower_horizontal_coverage = set((x, y) for x, y, z in bricks[other_brick]['blocks'])
        overlaps = lower_horizontal_coverage.intersection(horizontal_coverage)
        if len(overlaps) == 0:
            continue

        # Find the shortest fall possible
        shortest_fall = 1e100
        for overlap in overlaps:
            lowest_above = min(z for x, y, z in bricks[brick_name]['blocks'] if (x, y) == overlap)
            highest_below = max(z for x, y, z in bricks[other_brick]['blocks'] if (x, y) == overlap)

            if lowest_above - highest_below - 1 < shortest_fall:
                shortest_fall = lowest_above - highest_below - 1

        if fall is None or shortest_fall < fall[1]:
            # Store which brick(s) we land on, since this will be useful later
            fall = ([other_brick], shortest_fall)
        elif shortest_fall == fall[1]:
            fall[0].append(other_brick)
        

    if fall is not None:
        other_blocks, fall_length = fall
        bricks[brick_name]['blocks'] = [(x, y, z - fall_length) for x, y, z in bricks[brick_name]['blocks']]
        for other_block in other_blocks:
            bricks[brick_name]['supported_by'].append(other_block)
            bricks[other_block]['supporting'].append(brick_name)
    else:
        lowest_z = min(z for x, y, z in bricks[brick_name]['blocks'])
        fall_length = lowest_z - 1
        bricks[brick_name]['blocks'] = [(x, y, z - fall_length) for x, y, z in bricks[brick_name]['blocks']]

number_of_removable = 0
for brick_name in bricks:
    # Check that any brick that is supported by this brick has other bricks supporting it
    removable = True
    for supported_brick in bricks[brick_name]['supporting']:
        if len(bricks[supported_brick]['supported_by']) < 2:
            removable = False
            break

    if removable:
        number_of_removable += 1
print(f'There {number_of_removable} bricks that may be disintegrated')

def calculate_brick_fall(brick_name):
    # Calculate the lowest bricks first and work upwards, since it might not be apparent that a high brick will fall until all the
    # lower bricks under it have been removed
    falling = [brick_name]

    counter = 0
    while counter < len(falling):
        considered_brick = falling[counter]
        for supported_brick in bricks[considered_brick]['supporting']:
            should_fall = True
            for supported_by in bricks[supported_brick]['supported_by']:
                if supported_by not in falling:
                    should_fall = False
                    break
            if should_fall and supported_brick not in falling:
                falling.append(supported_brick)

        counter += 1

    # Remove one since the brick we're disintegrating (which is not one of the falling bricks) is also in this list
    return len(falling) - 1

brick_falls = []
for index, brick_name in enumerate(brick_names):
    brick_falls.append(calculate_brick_fall(brick_name))
print(f'The sum of all chain reactions is {sum(brick_falls)}')

There 473 bricks that may be disintegrated
The sum of all chain reactions is 61045


In [7]:
import sys
map = '''#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#'''.split('\n')
map = open('input23.txt', 'r').read().strip().split('\n')
start = (map[0].index('.'), 0)
finish = (map[-1].index('.'), len(map)-1)
directions = {'^': (0, -1), '<': (-1, 0), '>': (1, 0), 'v': (0, 1)}
woods = set((x, y) for y in range(len(map)) for x in range(len(map[y])) if map[y][x] == '#')
slopes = {(x,y): c for y in range(len(map)) for x, c in enumerate(map[y]) if c in directions}
height = len(map)
width = len(map[0])

sys.setrecursionlimit(height * width)

def find_neighbours(x, y, disregard_slopes):
    neighbours = set()
    neighbours_disregard_slopes = set()
    for direction in directions:
        slopes_disregarded = False
        if (x, y) in slopes and direction != slopes[(x, y)] and not disregard_slopes:
            continue
        dx, dy = directions[direction]
        nx = x+dx
        ny = y+dy
        if not (0 <= nx < width and 0 <= ny < height):
            continue
        neighbour = (nx, ny)
        if neighbour in woods:
            continue
        if neighbour in slopes and direction != slopes[neighbour] and not disregard_slopes:
            continue
        neighbours.add(neighbour)
    return neighbours

def create_graph(start, finish, disregard_slopes = False):
    nodes = {start: {'neighbour_nodes': [], 'explored_paths': set()}}

    node_iterator = 0
    while node_iterator < len(nodes):
        current_node_position = list(nodes.keys())[node_iterator]
        current_node = nodes[current_node_position]

        # Explore connections
        neighbours = find_neighbours(*current_node_position, disregard_slopes)

        for neighbour in neighbours:
            if neighbour in current_node['explored_paths']:
                continue
            current_node['explored_paths'].add(neighbour)

            previous_position = current_node_position
            current_position = neighbour
            distance = 1
            hit_slopes = current_position in slopes

            while len(current_neighbours := [
                position for position in find_neighbours(*current_position, disregard_slopes) 
                    if position != previous_position
            ]) == 1:
                distance += 1
                previous_position = current_position
                current_position = current_neighbours[0]

                if current_position in slopes:
                    hit_slopes = True

            # Found a splitting point
            if current_position not in nodes:
                nodes[current_position] = {'neighbour_nodes': [], 'explored_paths': set()}
            neighbour_node = nodes[current_position]
            current_node['neighbour_nodes'].append((current_position, distance))
            neighbour_node['explored_paths'].add(previous_position)
            if not hit_slopes or disregard_slopes:
                neighbour_node['neighbour_nodes'].append((current_node_position, distance))
            
        
        node_iterator += 1

    return nodes

slopes_graph = create_graph(start, finish)
no_slopes_graph = create_graph(start, finish, True)

def find_paths(graph, start, finish):
    path = {start: 0}

    def search(path):
        last_position = list(path.keys())[-1]
        distance_travelled = path[last_position]
        if last_position == finish:
            return [path]

        paths = []
        node = graph[last_position]

        for neighbour_node, neighbour_distance in node['neighbour_nodes']:
            if neighbour_node in path:
                continue

            path_copy = {position: distance for position, distance in path.items()}
            path_copy[neighbour_node] = distance_travelled + neighbour_distance
            for finished_path in search(path_copy):
                paths.append(finished_path)
        return paths

    return search(path)

directed_paths = find_paths(slopes_graph, start, finish)
print(f'The longest path is {max(path[finish] for path in directed_paths)}')

undirected_paths = find_paths(no_slopes_graph, start, finish)
print(f'The longest path with no slopes is {max(path[finish] for path in undirected_paths)}')

The longest path is 2294
The longest path with no slopes is 6418


In [28]:
import re
import numpy as np

hailstones_text = open('input24.txt', 'r').readlines()
hailstones = []

for hailstone in hailstones_text:
    x, y, z, vx, vy, vz = [int(x) for x in re.findall('([-]{0,1}[0-9]+)', hailstone)]
    hailstones.append(((x, y, z), (vx, vy, vz)))

def find_intersection(stone_one, stone_two):
    (x0, y0, z0), (vx0, vy0, vz0) = stone_one
    (x1, y1, z1), (vx1, vy1, vz1) = stone_two

    if vx0 == 0 and vx1 == 0:
        return None

    if vy0 == 0 and vy1 == 0:
        return None

    if vx0 == 0:
        s1 = (x0 - x1) / vx1
        s0 = (y1 + s1*vy1 - y0) / vy0
    elif vx1 == 0:
        s0 = (x1 - x0) / vx0
        s1 = (y0 + s0*vy0 - y1) / vy1
    elif vy0 == 0:
        s1 = (y0 - y1) / vy1
        s0 = (x1 + s1*vx1 - x0) / vx0
    elif vy1 == 0:
        s0 = (y1 - y0) / vy0
        s1 = (x0 + s0*vx0 - x1) / vx1
    elif abs(vy1 / vx1 - vy0 / vx0) < 0.001:
        return None
    else:
        try:
            s0 = (y1 - y0 + vy1 * (x0 - x1) / vx1) / (vy0 - vy1*vx0/vx1)
            s1 = (x0 + s0*vx0 - x1) / vx1
        except ZeroDivisionError:
            print(stone_one)
            print(stone_two)
            print()
            return None

    return (x0 + s0*vx0, y0 + s0*vy0, z0 + s0*vz0, s0, s1)

intersections_in_area = 0
for i in range(len(hailstones)):
    for j in range(i+1, len(hailstones)):
        stone0 = hailstones[i]
        stone1 = hailstones[j]
        intersection_test = find_intersection(stone0, stone1)
        if intersection_test is None:
            (x0, y0, z0), (vx0, vy0, vz0) = stone0
            (x1, y1, z1), (vx1, vy1, vz1) = stone1

            if abs((x0 - x1) / vx1 - (y0 - y1) / vy1) > 0.0001:
                continue

            # The lines lie on top of each other. Find out if a line goes inside the box
            x_enters = (200000000000000 - x0) / vx0
            x_leaves = (400000000000000 - x0) / vx0
            y_enters = (200000000000000 - y0) / vy0
            y_leaves = (400000000000000 - y0) / vy0

            if x_enters <= y_enters <= x_leaves or x_enters <= y_leaves <= x_leaves:
                intersections_in_area += 1
        else:
            x, y, z, s0, s1 = intersection_test
            if s0 < 0 or s1 < 0:
                continue
            if 200000000000000 <= x <= 400000000000000 and 200000000000000 <= y <= 400000000000000:
                intersections_in_area += 1

print(f'There are {intersections_in_area} intersections inside the specified area')

# The trajectory of the rock can be fully determined by finding where it intersects the trajectories
# of three other hailstones (the input will have to have been crafted so that every other hailstone also
# intersects that trajectory)

# Get the three first hailstones
(x0, y0, z0), (vx0, vy0, vz0) = hailstones[0]
(x1, y1, z1), (vx1, vy1, vz1) = hailstones[1]
(x2, y2, z2), (vx2, vy2, vz2) = hailstones[2]

# Use Newton's method to iteratively find a rock throw that goes through the first three hailstones
# The vector contains our current guesses for the following variables: x, y, z, vx, vy, vz, t0, t1, t2
# Where x, y, z are the start coordinates of the rock; vx, vy, vz is the start velocity;
# and t0, t1, t2 are the times of intersecting the first three hailstones

# Put in different values to avoid that the Jacobi matrix later becomes singular
guess = np.matrix([[1], [2], [3], [4], [5], [6], [7], [8], [9]])

function = np.matrix([
        [x - x0 + t0*(vx - vx0)],
        [y - y0 + t0*(vy - vy0)],
        [z - z0 + t0*(vz - vz0)],
        [x - x1 + t1*(vx - vx1)],
        [y - y1 + t1*(vy - vy1)],
        [z - z1 + t1*(vz - vz1)],
        [x - x2 + t2*(vx - vx2)],
        [y - y2 + t2*(vy - vy2)],
        [z - z2 + t2*(vz - vz2)]
    ])

while np.linalg.norm(function) > 1:
    x, y, z, vx, vy, vz, t0, t1, t2 = [guess.item((i, 0)) for i in range(9)]
    
    jacobi = np.matrix([
        [1, 0, 0, t0, 0, 0, vx - vx0, 0, 0],
        [0, 1, 0, 0, t0, 0, vy - vy0, 0, 0],
        [0, 0, 1, 0, 0, t0, vz - vz0, 0, 0],
        [1, 0, 0, t1, 0, 0, 0, vx - vx1, 0],
        [0, 1, 0, 0, t1, 0, 0, vy - vy1, 0],
        [0, 0, 1, 0, 0, t1, 0, vz - vz1, 0],
        [1, 0, 0, t2, 0, 0, 0, 0, vx - vx2],
        [0, 1, 0, 0, t2, 0, 0, 0, vy - vy2],
        [0, 0, 1, 0, 0, t2, 0, 0, vz - vz2]
    ])

    function = np.matrix([
        [x - x0 + t0*(vx - vx0)],
        [y - y0 + t0*(vy - vy0)],
        [z - z0 + t0*(vz - vz0)],
        [x - x1 + t1*(vx - vx1)],
        [y - y1 + t1*(vy - vy1)],
        [z - z1 + t1*(vz - vz1)],
        [x - x2 + t2*(vx - vx2)],
        [y - y2 + t2*(vy - vy2)],
        [z - z2 + t2*(vz - vz2)]
    ])

    # We know the solution has integer coordinates, because that's how the input must have been constructed
    update = np.around(np.matmul(np.linalg.inv(jacobi), function))
    guess = np.subtract(guess, update)

guess = guess.astype(int)
print(f'The sum of starting coordinates is {guess.item((0,0)) + guess.item((1,0)) + guess.item((2,0))}')

There are 13965 intersections inside the specified area
The sum of starting coordinates is 578177720733043


In [1]:
wiring = open('input25.txt').readlines()
components = {}
wires = []

for line in wiring:
    source, targets = line.split(':')

    if source.strip() not in components:
        components[source] = []

    for target in targets.split():
        target = target.strip()
        if target not in components:
            components[target] = []

        wire = [source, target]
        wire.sort()
        wire = tuple(wire)
        wires.append(wire)
        components[source].append(wire)
        components[target].append(wire)

def dijkstra(source, target, forbidden_edges=set()):
    candidates = {source: (0, [])}
    locked = {}

    while target not in locked and len(candidates) > 0:
        best_candidate = min([candidate for candidate in candidates], key=lambda c: candidates[c][0])
        distance, path = candidates[best_candidate]
        del candidates[best_candidate]
        locked[best_candidate] = (distance, path)

        connections = components[best_candidate]
        for connection in connections:
            if connection in forbidden_edges:
                continue
            neighbour = (connection[0] if connection[0] != best_candidate else connection[1])
            if neighbour in locked:
                continue
            elif neighbour in candidates and distance+1 >= candidates[neighbour][0]:
                continue
            path_copy = [p for p in path]
            path_copy.append(neighbour)
            candidates[neighbour] = (distance+1, path_copy)
    if target in locked:
        return locked[target]
    else:
        return None

# Calculate maximum flow to every other node. Solving the maximum flow problem is equivalent to finding the amount of edge-disjoint paths
source = list(components.keys())[0]
group_one = set([source])
group_two = set()
for index, target in enumerate(components):
    if source == target:
        continue
    forbidden_edges = set()
    edge_disjoint_paths = 0
    result = dijkstra(source, target, forbidden_edges)
    while result is not None:
        edge_disjoint_paths += 1
        distance, path = result
        path = [source] + path
        for i in range(1, len(path)):
            node_one, node_two = path[i-1], path[i]
            forbidden_edges.add(tuple(sorted([node_one, node_two])))
        result = dijkstra(source, target, forbidden_edges)
    print(f'\rSorted node {index+1} of {len(components)}', end='')
    if edge_disjoint_paths == 3:
        group_two.add(target)
    else:
        group_one.add(target)

print(f'\nGroup one has {len(group_one)} members. Group two has {len(group_two)}. Their product is {len(group_one)*len(group_two)}')

Sorted node 1552 of 1552
Group one has 771 members. Group two has 781. Their product is 602151


In [34]:
import cProfile
import time

maze = [[int(x) for x in line.strip()] for line in open('input17.txt', 'r').readlines()]
height = len(maze)
width = len(maze[0])


class NaiveNodeList:
    def __init__(self):
        self.nodes = {(0, 0, 0): 0, (0, 0, 1): 0}

    def remove_lowest_node(self):
        minimum = 1e10
        lowest_node = None

        for node in self.nodes:
            if self.nodes[node] < minimum:
                minimum = self.nodes[node]
                lowest_node = node

        del self.nodes[lowest_node]
        return lowest_node, minimum

    def add_node(self, node, cost):
        if node not in self.nodes or self.nodes[node] > cost:
            self.nodes[node] = cost


class LinkedListNode:
    __slots__ = 'coordinates', 'cost', 'next', 'previous'

    def __init__(self, coordinates, cost, next, previous):
        self.coordinates = coordinates
        self.cost = cost
        self.next = next
        self.previous = previous


class BetterNodeList:
    def __init__(self):
        last_node = LinkedListNode((0, 0, 0), 0, None, None)
        self.first_node = LinkedListNode((0, 0, 1), 0, last_node, None)
        last_node.previous = self.first_node
        self.nodes = {(0, 0, 0): last_node, (0, 0, 1): self.first_node}

    def remove_lowest_node(self):
        lowest = self.first_node
        self.remove_node(lowest)
        return lowest.coordinates, lowest.cost

    def remove_node(self, node):
        if node.previous is not None:
            node.previous.next = node.next
        else:
            self.first_node = node.next
        
        if node.next is not None:
            node.next.previous = node.previous

        del self.nodes[node.coordinates]

    def add_node(self, node, cost):    
        if node in self.nodes:
            old_node = self.nodes[node]
            if old_node.cost <= cost:
                return
            
            self.remove_node(old_node)
            search_direction = 'B'
            current_node = old_node.previous
            previous_node = old_node.next

        else:
            previous_node = None
            current_node = self.first_node
            search_direction = 'F'

        if search_direction == 'F':
            while current_node is not None and current_node.cost < cost:
                previous_node = current_node
                current_node = current_node.next
        elif search_direction == 'B':
            while current_node is not None and current_node.cost > cost:
                previous_node = current_node
                current_node = current_node.previous

            # We found the place to put our inserted node. Now swap the two
            temp = previous_node
            previous_node = current_node
            current_node = temp

        inserted_node = LinkedListNode(node, cost, current_node, previous_node)
        if previous_node is not None:
            previous_node.next = inserted_node
        else:
            self.first_node = inserted_node

        if current_node is not None:
            current_node.previous = inserted_node

        self.nodes[node] = inserted_node

def find_neighbours(node, min_length, max_length):
    directions = [(1,0), (0, 1), (-1, 0), (0, -1)]
    costs = [0, 0, 0, 0]
    neighbours = {}

    # Nodes are stored as their coordinates and the direction you are allowed to go in from them. This direction restriction is stored
    # as either 0 or 1. The advantage of this is that in a list containing [RIGHT, UP, LEFT, DOWN], the index of the horizontal directions
    # modulo 2 is equal to zero, and the index of the vertical directions modulo 2 is equal to 1
    x, y, dir_restriction = node
    for i in range(1, max_length + 1):
        for dir_index, (dx, dy) in enumerate(directions):
            # As stated earlier, the index of the direction modulo 2 tells us if it is horizontal or vertical
            if dir_index % 2 != dir_restriction:
                continue
            cx, cy = x + i*dx, y + i*dy
            if not (0 <= cx < width and 0 <= cy < height):
                continue
            costs[dir_index] += maze[cy][cx]

            if i >= min_length:
                # The direction restriction of the neighbour must be opposite of the current node. If we went horizontally to get there, we
                # need to leave the neighbour vertically
                neighbours[(cx, cy, (dir_restriction + 1) % 2)] = costs[dir_index]
    return neighbours


def dijkstra(node_list, target_coordinates, min_length, max_length):
    tx, ty = target_coordinates
    locked = {}
    while (tx, ty, 0) not in locked and (tx, ty, 1) not in locked:
        best_node, cost = node_list.remove_lowest_node()
        locked[best_node] = cost
        neighbours = find_neighbours(best_node, min_length, max_length)
        for neighbour in neighbours:
            if neighbour in locked:
                continue
            neighbour_cost = cost + neighbours[neighbour]
            node_list.add_node(neighbour, neighbour_cost)
            
    return min([locked[node] for node in [(tx, ty, 0), (tx, ty, 1)] if node in locked])


target_coordinates = (width-1, height-1)
start_time = time.process_time()
naive_list = NaiveNodeList()
first_problem = dijkstra(naive_list, target_coordinates, 1, 3)
print(f'The solution of the first part is {first_problem}')

naive_list = NaiveNodeList()
second_problem = dijkstra(naive_list, target_coordinates, 4, 10)
print(f'The solution of the second part is {second_problem}')
end_time = time.process_time()
print(f'The time taken with the naive approach is {end_time - start_time}')
print()

start_time = time.process_time()
better_list = BetterNodeList()
first_problem = dijkstra(better_list, target_coordinates, 1, 3)
print(f'The solution of the first part is {first_problem}')

better_list = BetterNodeList()
second_problem = dijkstra(better_list, target_coordinates, 4, 10)
print(f'The solution of the second part is {second_problem}')
end_time = time.process_time()
print(f'The time taken with the linked list approach is {end_time - start_time}')

The solution of the first part is 665
The solution of the second part is 809
The time taken with the naive approach is 10.984375

The solution of the first part is 665
The solution of the second part is 809
The time taken with the linked list approach is 6.609375
